In [29]:
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from model_script.script_XGBoost_model_polars import DataCleaner, FeatureCreator, AnomalyFilter, WeightedPOICountsTransformer, data_loader, build_xgboost_model,save_model,plot_train_test_predictions

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# YEARS = ["2019","2020", "2021", "2022", "2023","2024"]
# PREPROCESSED_FILE = "data_processed/data_dvf_preprocessed.csv"
# OVERWRITE = True

# print("Starting preprocessing by year...")

# # Charger la grille pour le pipeline
# df_grid = pl.read_csv('data_pop_density/dataframe_densite&amenities_radius=500.csv')

# # Définir le pipeline de preprocessing
# pipeline_preprocess = Pipeline(steps=[
#     ("cleaner", DataCleaner(nombre_lots_max=1, cutoff_valeur_fonciere_min=1e5, cutoff_valeur_fonciere_max=10e6, min_surface=15)),
#     ("feature_creator", FeatureCreator()),
#     ("anomaly_filter", AnomalyFilter(contamination=0.5, target_elimination=True)),
#     ('weighted_poi', WeightedPOICountsTransformer(n_neighbors=4)),
# ])

# pipeline_preprocess.set_params(weighted_poi__df_grid=df_grid)

# # Effacer le fichier CSV s'il existe déjà (évite d'ajouter plusieurs fois les mêmes données)
# if OVERWRITE:
#     with open(PREPROCESSED_FILE, "w") as f:
#         pass

#     # === 1. Charger et prétraiter chaque année séparément ===
#     for year in YEARS:
#         print(f"Processing year {year}...")

#         # Charger les données de l'année en cours
#         df_year = data_loader('data_dvf', annees=[year])
#         print(f"Data loaded for {year}: {df_year.shape[0]} rows")

#         # Appliquer le préprocessing
#         df_processed = pipeline_preprocess.fit_transform(df_year)
#         print(f"Data processed for {year}: {df_processed.shape[0]} rows")

#         # Sauvegarder les données prétraitées en ajoutant au CSV
#         with open(PREPROCESSED_FILE, mode="a") as f:
#             df_processed.write_csv(f,include_header=True)

#     print("All years processed and saved!")

# # === 2. Charger toutes les données prétraitées une fois que tout est stocké ===
# df_final = pl.read_csv(PREPROCESSED_FILE,columns=[
#                                     'surface_reelle_bati', 'year','type_local', 'sin_month', 'cos_month', 'nombre_lots',
#                                     'total_surface_carrez', 'lot1_surface_carrez', 'lot2_surface_carrez',
#                                     'lot3_surface_carrez', 'lot4_surface_carrez', 'lot5_surface_carrez',
#                                     'nombre_pieces_principales', 'surface_terrain', 'longitude', 'latitude','valeur_fonciere',
#                                     'densite_weighted', 'transport_pois_weighted', 'education_pois_weighted',
#                                     'health_pois_weighted', 'food_pois_weighted', 'shopping_pois_weighted',
#                                     'park_pois_weighted', 'entertainment_pois_weighted', 'cultural_pois_weighted'
#                                     ],
#                                 schema_overrides={
#                                         'surface_reelle_bati':pl.Float32,
#                                         'type_local':pl.Utf8,
#                                         'year':pl.Float32,
#                                         'sin_month':pl.Float32,
#                                         'cos_month':pl.Float32,
#                                         'nombre_lots':pl.Float32,
#                                         'total_surface_carrez':pl.Float32,
#                                         'lot1_surface_carrez':pl.Float32,
#                                         'lot2_surface_carrez':pl.Float32,
#                                         'lot3_surface_carrez':pl.Float32,
#                                         'lot4_surface_carrez':pl.Float32,
#                                         'lot5_surface_carrez':pl.Float32,
#                                         'nombre_pieces_principales':pl.Float32,
#                                         'surface_terrain':pl.Float32,
#                                         'longitude':pl.Float32,
#                                         'latitude':pl.Float32,
#                                         'valeur_fonciere':pl.Float32,
#                                         'densite_weighted':pl.Float32,
#                                         'transport_pois_weighted':pl.Float32,
#                                         'education_pois_weighted':pl.Float32,
#                                         'health_pois_weighted':pl.Float32,
#                                         'food_pois_weighted':pl.Float32,
#                                         'shopping_pois_weighted':pl.Float32,
#                                         'park_pois_weighted':pl.Float32,
#                                         'entertainment_pois_weighted':pl.Float32,
#                                         'cultural_pois_weighted':pl.Float32
#                                     },
#                                 ignore_errors=True).drop_nulls()
# print(f"Final dataset loaded: {df_final.shape[0]} rows")

# # === 3. Split Train/Test ===
# X = df_final.drop('valeur_fonciere')
# y = df_final['valeur_fonciere']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# # === 4. Pipeline d'encodage ===
# categorical_columns_onehot = ['type_local']
# numerical_columns = [
#     'surface_reelle_bati', 'year', 'sin_month', 'cos_month', 'nombre_lots',
#     'total_surface_carrez', 'lot1_surface_carrez', 'lot2_surface_carrez',
#     'lot3_surface_carrez', 'lot4_surface_carrez', 'lot5_surface_carrez',
#     'nombre_pieces_principales', 'surface_terrain', 'longitude', 'latitude',
#     'densite_weighted', 'transport_pois_weighted', 'education_pois_weighted',
#     'health_pois_weighted', 'food_pois_weighted', 'shopping_pois_weighted',
#     'park_pois_weighted', 'entertainment_pois_weighted', 'cultural_pois_weighted'
# ]
# unique_categories = [X[col].drop_nulls().unique() for col in categorical_columns_onehot]
# # Créer les catégories pour OneHotEncoder
# # === 5. Entraînement du modèle ===
# xgb_model = build_xgboost_model(numerical_columns, categorical_columns_onehot, unique_categories )
# xgb_model.fit(X_train, y_train)

# # === 6. Sauvegarde du modèle ===
# save_model(xgb_model, 'result_xgb_model/xgboost_model.pkl')

# # === 7. Visualisation des prédictions ===
# plot_train_test_predictions(
#     y_train=y_train,
#     y_test=y_test,
#     X_train=X_train,
#     X_test=X_test,
#     model=xgb_model,
#     save_path="result_xgb_model/train_test_scatter_plots_with_metrics.png"
# )


In [31]:
df_year = data_loader('data_dvf', annees=['2019'])
df_year.shape

(1967866, 15)

In [32]:
dc = DataCleaner()
df_year_2=dc.fit_transform(df_year)

Shape before filtering: (1967866, 15)
Shape after filtering: (1883128, 15)


In [5]:
import polars as pl
df_test = pl.read_csv('data_processed/data_dvf_preprocessed.csv').head(10000)
df_test.describe()

ComputeError: could not parse `valeur_fonciere` as dtype `f64` at column 'valeur_fonciere' (column number 1)

The current offset in the file is 5935342 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `valeur_fonciere` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [34]:
df_test = pl.DataFrame({
    'id_mutation': [1, 2],
    'date_mutation': ['2023-01-01', '2023-01-02'],
    'type_local': ['Maison', 'Appartement'],
    'surface_reelle_bati': [50, 70],
    'nombre_lots': [2, 3],
    'lot1_surface_carrez': [30, 40],
    'lot2_surface_carrez': [20, 30],
    'lot3_surface_carrez': [0, 0],
    'lot4_surface_carrez': [0, 0],
    'lot5_surface_carrez': [0, 0],
    'nombre_pieces_principales': [5, 6],
    'surface_terrain': [200, 300],
    'longitude': [2.35, 2.36],
    'latitude': [48.85, 48.86],
    'valeur_fonciere': [200000, 250000]
})

dc = DataCleaner()
df_test_transformed = dc.fit_transform(df_test)
print(df_test_transformed)


Shape before filtering: (2, 15)
Shape after filtering: (2, 15)
shape: (2, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ id_mutati ┆ date_muta ┆ type_loca ┆ surface_r ┆ … ┆ surface_t ┆ longitude ┆ latitude ┆ valeur_fo │
│ on        ┆ tion      ┆ l         ┆ eelle_bat ┆   ┆ errain    ┆ ---       ┆ ---      ┆ nciere    │
│ ---       ┆ ---       ┆ ---       ┆ i         ┆   ┆ ---       ┆ f64       ┆ f64      ┆ ---       │
│ i64       ┆ str       ┆ str       ┆ ---       ┆   ┆ i64       ┆           ┆          ┆ i64       │
│           ┆           ┆           ┆ i64       ┆   ┆           ┆           ┆          ┆           │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 1         ┆ 2023-01-0 ┆ Maison    ┆ 50        ┆ … ┆ 200       ┆ 2.35      ┆ 48.85    ┆ 200000    │
│           ┆ 1         ┆           ┆           ┆   ┆           ┆           ┆          ┆           │
│ 2         ┆